In [1]:
#Common Imports
import tensorflow as tf
import tensorflow.keras as keras

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

In [2]:
#Image Data Generator manipulates and "augments" images

train_datagen = ImageDataGenerator(rescale=1/255)


# Directory Iterator reads images from a directory

train_data = DirectoryIterator(
    directory="LEGO/train",
    image_data_generator = train_datagen,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)

Found 6379 images belonging to 16 classes.


In [3]:
valid_datagen = ImageDataGenerator(rescale=1/255)

valid_data = DirectoryIterator(
    directory="LEGO/valid",
    image_data_generator = valid_datagen,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)

Found 1555 images belonging to 16 classes.


In [4]:
# Building Model

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(50,50,1)))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))


model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(16, activation='softmax'))

model.summary()

# Initiating adam optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# Training the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 11, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0

In [5]:
model.fit(
        valid_data,
        steps_per_epoch=45,
        epochs=10,
        validation_data=valid_data,
        validation_steps=20
        )

Epoch 1/10
45/45 [==============================] - 4s 71ms/step - loss: 2.5391 - accuracy: 0.1866 - val_loss: 1.2379 - val_accuracy: 0.5953
Epoch 2/10
45/45 [==============================] - 3s 61ms/step - loss: 1.4575 - accuracy: 0.4839 - val_loss: 0.8708 - val_accuracy: 0.6859
Epoch 3/10
45/45 [==============================] - 3s 63ms/step - loss: 1.1389 - accuracy: 0.5722 - val_loss: 0.7326 - val_accuracy: 0.7500
Epoch 4/10
45/45 [==============================] - 3s 60ms/step - loss: 1.0570 - accuracy: 0.6146 - val_loss: 0.6099 - val_accuracy: 0.7875
Epoch 5/10
45/45 [==============================] - 3s 62ms/step - loss: 0.9494 - accuracy: 0.6400 - val_loss: 0.5981 - val_accuracy: 0.7734
Epoch 6/10
45/45 [==============================] - 3s 61ms/step - loss: 0.8485 - accuracy: 0.6678 - val_loss: 0.5168 - val_accuracy: 0.8234
Epoch 7/10
45/45 [==============================] - 3s 61ms/step - loss: 0.8208 - accuracy: 0.6879 - val_loss: 0.5136 - val_accuracy: 0.8531
Epoch 8/10
45

In [6]:
# Evaluating the model

scores = model.evaluate(valid_data, verbose=0)

scores

# Extracting the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.38
accuracy: 86.62%
